In [1]:
import pandas as pd 

books = pd.read_csv("books_with_categories.csv")

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification" ,
                    model="j-hartmann/emotion-english-distilroberta-base" ,
                    top_k = None ,
                    device="mps")

classifier("I love this")

Device set to use mps


[[{'label': 'joy', 'score': 0.9845667481422424},
  {'label': 'surprise', 'score': 0.004927210509777069},
  {'label': 'sadness', 'score': 0.004531432408839464},
  {'label': 'neutral', 'score': 0.0034753207582980394},
  {'label': 'anger', 'score': 0.0013875329168513417},
  {'label': 'disgust', 'score': 0.0007134057814255357},
  {'label': 'fear', 'score': 0.0003984913637395948}]]

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.654841423034668},
  {'label': 'neutral', 'score': 0.16985176503658295},
  {'label': 'sadness', 'score': 0.11640898138284683},
  {'label': 'surprise', 'score': 0.020700616762042046},
  {'label': 'disgust', 'score': 0.019100748002529144},
  {'label': 'joy', 'score': 0.01516124326735735},
  {'label': 'anger', 'score': 0.003935149405151606}]]

In [10]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [11]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [12]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.14038553833961487},
 {'label': 'fear', 'score': 0.0681622102856636},
 {'label': 'joy', 'score': 0.047942448407411575},
 {'label': 'anger', 'score': 0.009156349115073681},
 {'label': 'disgust', 'score': 0.002628474263474345},
 {'label': 'sadness', 'score': 0.002122161677107215}]

In [14]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [15]:
predictions[3]

[{'label': 'fear', 'score': 0.9281687140464783},
 {'label': 'anger', 'score': 0.03219045326113701},
 {'label': 'neutral', 'score': 0.012808630242943764},
 {'label': 'sadness', 'score': 0.0087568499147892},
 {'label': 'surprise', 'score': 0.00859786756336689},
 {'label': 'disgust', 'score': 0.008431782945990562},
 {'label': 'joy', 'score': 0.001045578857883811}]

In [16]:
sorted(predictions[0] , key=lambda x : x['label'])

[{'label': 'anger', 'score': 0.009156349115073681},
 {'label': 'disgust', 'score': 0.002628474263474345},
 {'label': 'fear', 'score': 0.0681622102856636},
 {'label': 'joy', 'score': 0.047942448407411575},
 {'label': 'neutral', 'score': 0.14038553833961487},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296027541160583}]

In [17]:
import numpy as np

emotion_labels = ["anger" , "disgust" , "fear" , "joy" , "sadness" , "surprise" , "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction , key = lambda x : x["label"])
        for index , label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])

    return {label: np.max(scores) for label , scores in per_emotion_scores.items()} 


In [18]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [19]:
emotion_scores

{'anger': [0.06413355469703674,
  0.6126182675361633,
  0.06413355469703674,
  0.3514835238456726,
  0.08141201734542847,
  0.23222509026527405,
  0.538184404373169,
  0.06413355469703674,
  0.30067023634910583,
  0.06413355469703674],
 'disgust': [0.2735903859138489,
  0.34828686714172363,
  0.10400659590959549,
  0.1507226973772049,
  0.18449506163597107,
  0.7271745800971985,
  0.1558549702167511,
  0.10400659590959549,
  0.27948078513145447,
  0.17792750895023346],
 'fear': [0.9281687140464783,
  0.9425276517868042,
  0.9723208546638489,
  0.360706627368927,
  0.09504329413175583,
  0.05136273801326752,
  0.7474284172058105,
  0.404496431350708,
  0.9155242443084717,
  0.05136273801326752],
 'joy': [0.9327981472015381,
  0.7044217586517334,
  0.7672364115715027,
  0.2518806755542755,
  0.04056435823440552,
  0.04337591305375099,
  0.8725653886795044,
  0.04056435823440552,
  0.04056435823440552,
  0.04056435823440552],
 'sadness': [0.6462160348892212,
  0.8879396319389343,
  0.5494

In [20]:
from tqdm import tqdm

emotion_labels = ["anger" , "disgust" , "fear" , "joy" , "sadness" , "surprise" , "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [07:34<00:00, 11.44it/s]


In [21]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn
emotions_df

anger   disgust      fear       joy   sadness  surprise   neutral  \
0     0.064134  0.273590  0.928169  0.932798  0.646216  0.967158  0.729603   
1     0.612618  0.348287  0.942528  0.704422  0.887940  0.111690  0.252545   
2     0.064134  0.104007  0.972321  0.767236  0.549477  0.111690  0.078765   
3     0.351484  0.150723  0.360707  0.251881  0.732686  0.111690  0.078765   
4     0.081412  0.184495  0.095043  0.040564  0.884389  0.475881  0.078765   
...        ...       ...       ...       ...       ...       ...       ...   
5192  0.148208  0.030643  0.919165  0.255169  0.853722  0.980877  0.030656   
5193  0.064134  0.114383  0.051363  0.400263  0.883198  0.111690  0.227765   
5194  0.009997  0.009929  0.339217  0.947779  0.375756  0.066685  0.057625   
5195  0.064134  0.104007  0.459269  0.759455  0.951104  0.368110  0.078765   
5196  0.064134  0.104007  0.051363  0.958549  0.915193  0.111690  0.078765   

             isbn13  
0     9780002005883  
1     9780002261982  
2     9780006178736  
3     9780006280897  
4     9780006280934  
...             ...  
5192  9788172235222  
5193  9788173031014  
5194  9788179921623  
5195  9788185300535  
5196  9789027712059  

[5197 rows x 8 columns]

In [22]:
books = pd.merge(books , emotions_df , on = "isbn13")

In [23]:
books

isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5192  9788172235222  8172235224   
5193  9788173031014  8173031010   
5194  9788179921623  817992162X   
5195  9788185300535  8185300534   
5196  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5192                                  Mistaken Identity   
5193                                Journey to the East   
5194  The Monk Who Sold His Ferrari: A Fable About F...   
5195                                          I Am that   
5196                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5192                              Nayantara Sahgal   
5193                                 Hermann Hesse   
5194                                  Robin Sharma   
5195  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5196                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5192        Indic fiction (English)   
5193              Adventure stories   
5194               Health & Fitness   
5195                     Philosophy   
5196                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5192  http://books.google.com/books/content?id=q-tKP...   
5193  http://books.google.com/books/content?id=rq6JP...   
5194  http://books.google.com/books/content?id=c_7mf...   
5195  http://books.google.com/books/content?id=Fv_JP...   
5196  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5192  On A Train Journey Home To North India After L...          2003.0   
5193  This book tells the tale of a man who goes on ...          2002.0   
5194  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5195  This collection of the timeless teachings of o...          1999.0   
5196  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ...  \
0               3.85      247.0  ...   
1               3.83      241.0  ...   
2               3.93      512.0  ..

In [24]:
books.to_csv("books_with_emotions.csv" , index=False)